# B25 - Training

In [1]:
from gensim.models import Word2Vec
import pickle

In [2]:
with open("../data/tokenized_data/playlist_names/dataset_train_v2.pkl", "rb") as f:
    train_dataset = pickle.load(f)

In [ ]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=50, window=5, min_count=1, sg=0
)
word2vec_model.save("../models/b25-sn-v50/b25-sn-v50.model")

In [ ]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=256, window=5, min_count=1, sg=0
)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-a.model")

In [ ]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=256, window=10, min_count=1, sg=0
)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-b.model")

In [5]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=256, window=20, min_count=1, sg=0
)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-c-a.model")

In [3]:
word2vec_model = Word2Vec(
    sentences=train_dataset,
    workers=10,
    vector_size=256,
    window=20,
    min_count=1,
    sg=1,
    ns_exponent=0.0,
    epochs=40,  # prev 5 than 10 than 20
)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-d-c.model")

In [ ]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=512, window=100, min_count=1, sg=0
)
word2vec_model.save("../models/b25-sn-v512/b25-sn-v512-a.model")

In [3]:
word2vec_model = Word2Vec(
    sentences=train_dataset,
    workers=10,
    vector_size=512,
    window=100,
    min_count=1,
    sg=1,
    ns_exponent=0.0,
    epochs=5,
)
word2vec_model.save("../models/b25-sn-v512/b25-sn-v512-b-b.model")

In [3]:
word2vec_model = Word2Vec(
    sentences=train_dataset,
    workers=10,
    vector_size=2048,
    window=100,
    min_count=1,
    sg=1,
    ns_exponent=0.0,
)
word2vec_model.save("../models/b25-sn-v2048/b25-sn-v2048-a.model")

In [ ]:
from algo_testing import song2vec_c

model = song2vec_c.Song2VecC(
    train_dataset, vector_size=512, epochs=15, learning_rate=0.015
)

with open("../models/b25-sn-v512/b25-sn-v512-o.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
from algo_testing import song2vec_c

model = song2vec_c.Song2VecC(
    train_dataset, vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-d.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
# testing with additional score
# the issues is the cluster forming is mostly towards the middle of the box with a varying differentiation of the things
# testing litte teaks but i think there is still the need to pivot a bit more

from algo_testing import song2vec_d

model = song2vec_d.Song2VecD(
    train_dataset, vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-e.pkl", "wb") as f:
    pickle.dump(model, f)
# training was ok took the normal amount. Could use the squard L2 Norm

In [ ]:
from algo_testing import son2vecgl_a

model = son2vecgl_a.Song2GloveA(
    training_data=train_dataset[0:1000], vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-g.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
len(train_dataset)

In [ ]:
from algo_testing import son2vecgl_b

model = son2vecgl_b.Song2GloveB(
    training_data=train_dataset[0:1000], vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-h.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
from algo_testing import son2vecgl_c

model = son2vecgl_c.Song2GloveC(
    training_data=train_dataset, vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-i.pkl", "wb") as f:
    pickle.dump(model, f)

## Param

#### b25-sn-v50
> vector_size=50, window=5, min_count=1, sg=0

#### b25-sn-v256-a
> vector_size=256, window=5, min_count=1, sg=0

#### b25-sn-v256-b
> vector_size=256, window=10, min_count=1, sg=0

#### b25-sn-v256-c
> vector_size=256, window=20, min_count=1, sg=0

#### b25-sn-v256-d
> vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0

#### b25-sn-v512-a
> vector_size=512, window=100, min_count=1, sg=0

#### b25-sn-v512-b
> vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0

#### b25-sn-v512-c - CBOS
> vector_size=512, epochs=15, learning_rate=0.015

#### b25-sn-v512-d - CBOS
> vector_size=512, epochs=15, learning_rate=0.025